# 원, 삼각형, 사각형 구분하기

## 3. Data Augmentation을 사용한 CNN 모델

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
seed = 2020
np.random.seed(seed)
tf.random.set_seed(seed)

## 데이터 부풀리기(Augmentation)

In [3]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img

In [5]:
# 데이터 부풀리기를 적용한 훈련 데이터 제너레이터
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range =15,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.5,
                                  zoom_range = [0.8, 0.2],
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  fill_mode='nearest')

In [8]:
img = load_img('hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0

for batch in train_datagen.flow(x, batch_size = 1, save_to_dir='preview', save_prefix = 'tri', save_format = 'png'):
    i +=1
    if i > 30:
        break

## 데이터셋 생성

In [9]:
train_generator = train_datagen.flow_from_directory(
        'hard_handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'hard_handwriting_shape/test',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


## 모델 구성

In [10]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               819328    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
________________________________________________

## 모델 학습과정 설정

In [11]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])


## 모델 학습

- 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 train_generator으로 지정.
- steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정.
- epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정. 200번을 반복적으로 학습.
- validation_data : 검증데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 validation_generator으로 지정.
- validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정. 홍 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정.

In [12]:
model.fit_generator(
        train_generator,
        steps_per_epoch=15,   #원래는 *100 해줘야함(시간이 오래걸리니 생략)
        epochs=200,
        validation_data=test_generator,
        validation_steps=5)

Epoch 1/200
15/15 [==============================] - 0s 18ms/step - loss: 1.3679 - accuracy: 0.4222 - val_loss: 1.2079 - val_accuracy: 0.3333
Epoch 2/200
15/15 [==============================] - 0s 17ms/step - loss: 1.1083 - accuracy: 0.4000 - val_loss: 1.0758 - val_accuracy: 0.3333
Epoch 3/200
15/15 [==============================] - 0s 21ms/step - loss: 1.0482 - accuracy: 0.3778 - val_loss: 1.3465 - val_accuracy: 0.3333
Epoch 4/200
15/15 [==============================] - 0s 21ms/step - loss: 0.9745 - accuracy: 0.3778 - val_loss: 1.0505 - val_accuracy: 0.3333
Epoch 5/200
15/15 [==============================] - 0s 20ms/step - loss: 0.9445 - accuracy: 0.4667 - val_loss: 4.5883 - val_accuracy: 0.3333
Epoch 6/200
15/15 [==============================] - 0s 21ms/step - loss: 0.9273 - accuracy: 0.5111 - val_loss: 2.4016 - val_accuracy: 0.3333
Epoch 7/200
15/15 [==============================] - 0s 20ms/step - loss: 0.7918 - accuracy: 0.6667 - val_loss: 7.3831 - val_accuracy: 0.3333
Epoch 

15/15 [==============================] - 0s 21ms/step - loss: 0.8905 - accuracy: 0.6000 - val_loss: 7.1904 - val_accuracy: 0.3333
Epoch 59/200
15/15 [==============================] - 0s 20ms/step - loss: 0.6099 - accuracy: 0.6889 - val_loss: 7.1127 - val_accuracy: 0.3333
Epoch 60/200
15/15 [==============================] - 0s 21ms/step - loss: 0.5525 - accuracy: 0.7556 - val_loss: 9.8591 - val_accuracy: 0.3333
Epoch 61/200
15/15 [==============================] - 0s 20ms/step - loss: 0.5169 - accuracy: 0.7778 - val_loss: 7.2005 - val_accuracy: 0.3333
Epoch 62/200
15/15 [==============================] - 0s 21ms/step - loss: 0.6053 - accuracy: 0.6667 - val_loss: 13.0109 - val_accuracy: 0.3333
Epoch 63/200
15/15 [==============================] - 0s 21ms/step - loss: 0.4911 - accuracy: 0.7778 - val_loss: 3.8903 - val_accuracy: 0.3333
Epoch 64/200
15/15 [==============================] - 0s 21ms/step - loss: 0.5320 - accuracy: 0.6889 - val_loss: 11.1028 - val_accuracy: 0.3333
Epoch 65/2

15/15 [==============================] - 0s 20ms/step - loss: 0.4491 - accuracy: 0.8000 - val_loss: 4.1857 - val_accuracy: 0.4000
Epoch 116/200
15/15 [==============================] - 0s 20ms/step - loss: 0.5807 - accuracy: 0.6889 - val_loss: 5.4102 - val_accuracy: 0.4000
Epoch 117/200
15/15 [==============================] - 0s 21ms/step - loss: 0.4046 - accuracy: 0.7556 - val_loss: 4.4053 - val_accuracy: 0.4000
Epoch 118/200
15/15 [==============================] - 0s 21ms/step - loss: 0.5432 - accuracy: 0.7333 - val_loss: 11.6860 - val_accuracy: 0.3333
Epoch 119/200
15/15 [==============================] - 0s 20ms/step - loss: 0.4838 - accuracy: 0.8000 - val_loss: 5.6897 - val_accuracy: 0.4000
Epoch 120/200
15/15 [==============================] - 0s 20ms/step - loss: 0.4129 - accuracy: 0.7778 - val_loss: 6.2079 - val_accuracy: 0.3333
Epoch 121/200
15/15 [==============================] - 0s 20ms/step - loss: 0.5854 - accuracy: 0.7333 - val_loss: 1.6700 - val_accuracy: 0.4000
Epoch

Epoch 172/200
15/15 [==============================] - 0s 19ms/step - loss: 0.3820 - accuracy: 0.8889 - val_loss: 1.1252 - val_accuracy: 0.4667
Epoch 173/200
15/15 [==============================] - 0s 21ms/step - loss: 0.4963 - accuracy: 0.7556 - val_loss: 8.6725 - val_accuracy: 0.4667
Epoch 174/200
15/15 [==============================] - 0s 21ms/step - loss: 0.3531 - accuracy: 0.8444 - val_loss: 7.0303 - val_accuracy: 0.4667
Epoch 175/200
15/15 [==============================] - 0s 18ms/step - loss: 0.4019 - accuracy: 0.8000 - val_loss: 4.7856 - val_accuracy: 0.4667
Epoch 176/200
15/15 [==============================] - 0s 21ms/step - loss: 0.3752 - accuracy: 0.8000 - val_loss: 5.8061 - val_accuracy: 0.4667
Epoch 177/200
15/15 [==============================] - 0s 19ms/step - loss: 0.4598 - accuracy: 0.7333 - val_loss: 5.4104 - val_accuracy: 0.4667
Epoch 178/200
15/15 [==============================] - 0s 21ms/step - loss: 0.3698 - accuracy: 0.8444 - val_loss: 2.8874 - val_accuracy:

## 모델 평가하기

In [13]:
print('--Evaluate--')
scores = model.evaluate_generator(test_generator, steps = 5)
print('%s: %.2f%%' %(model.metrics_names[1], scores[1]*100))

--Evaluate--
accuracy: 40.00%
